# Explore here

K Vecinos Más Cercanos
      
      Objetivo: Predecir qué películas podrían ser un éxito comercial.

In [1]:
!pip install missingno lazypredict
!pip install pandas sqlite3


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [26]:
import pandas as pd
import numpy as np
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

1. Cargar el conjunto de datos.

In [3]:
# Descargo los datos base datos "movies":

# Leo los datos de la URL.
url = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv"

# Guardo los datos en un DataFrame. La BD tiene 4803 filas y 20 columnas.
df_movies = pd.read_csv(url)

# Miro las variables de la bd.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
# Descargo los datos base datos "credits":

# Leo los datos de la URL.
url = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv"

# Guardo los datos en un DataFrame. La bd contiene 4803 filas y 4 columnas.
df_credits = pd.read_csv(url)

# Miro las variables de la bd.
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


2. Creación de una base de datos.

In [5]:
# Creo una bd e inserto df_movies y df_credits como tablas.

# Creo una conexión a la base de datos SQLite
conn = sqlite3.connect('bd_moviesycredits.db')

# Guardo los df en la base de datos como tablas
df_movies.to_sql('movies', conn, if_exists='replace', index=False)
df_credits.to_sql('credits', conn, if_exists='replace', index=False)

# Verifico que las tablas se han guardado correctamente
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('movies',), ('credits',)]


In [6]:
# Uno las tablas en SQL
query = '''
SELECT m.*, c.cast, c.crew, c.movie_id 
FROM movies m
JOIN credits c
ON m.title = c.title;
'''

# Ejecuto la consulta y guardo el resultado en un DataFrame (df_mc)
df_mc = pd.read_sql(query, conn)

# Muestro las primeras filas de la tabla unificada
print(df_mc.head())

df_mc.info()

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [7]:
# Limpio el df_mv y me quedo con "movie_id", "title", "overview", "genres", "keywords", "cast", "crew".

df_mc= df_mc[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

df_mc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


3. Transformar los datos.

In [8]:
# Miro el formato de las columnas para saber cuáles tienen formato JSON.

def es_json(val):
    try:
        json.loads(val)
        return True
    except ValueError:
        return False

# Aplicar la función a la columna del DataFrame
df_mc['es_json'] = df_mc['crew'].apply(es_json)

print(df_mc)

      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                               overview  \
0     In the 22nd century, a paraplegic Marine is di...   
1     Captain Barbossa, long believed to be dead, ha...   
2     A cryptic message from Bond’s past sends him o...   
3     Following the death of District Attor

Las variables JSON son genres, keywords, cast y crew.

In [9]:
# Selecciono el atributo name de las variables JSON.
def extract_names(json_column):
    try:
        # Convierto el JSON a lista de diccionarios
        items = json.loads(json_column)
        # Extraigo todos los nombres
        return [item['name'] for item in items]
    except (TypeError, json.JSONDecodeError):
        return []

In [10]:
# Selecciono los tres primeros nombres en 'cast'.
def extract_top_cast(json_column):
    try:
        items = json.loads(json_column)
        # Extraigo los tres primeros nombres
        return [item['name'] for item in items[:3]]  # Limita a los tres primeros
    except (TypeError, json.JSONDecodeError):
        return []


In [11]:
# Aplico las funciones a las columnas correspondientes.
df_mc['genres'] = df_mc['genres'].apply(extract_names)
df_mc['keywords'] = df_mc['keywords'].apply(extract_names)
df_mc['cast'] = df_mc['cast'].apply(extract_top_cast)

In [12]:
print(df_mc[['genres', 'keywords', 'cast']].head())

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast  
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley]  
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]  
3      [Christian Bale, Michael Caine, Gary Oldman]  
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]  


In [13]:
# Selecciono el nombre del director de la columna 'crew'
def extract_director(json_column):
    try:
        # Convierto la columna 'crew' en una lista de diccionarios
        crew_list = json.loads(json_column)
        # Busco la persona cuyo rol es "Director"
        for person in crew_list:
            if person.get('job') == 'Director':
                return person.get('name')
        return None  # Si no se encuentra un director, devuelve None
    except (TypeError, json.JSONDecodeError):
        return None

In [14]:
# Convierto el 'overview' en una lista de palabras
def overview_to_list(text_column):
    try:
        return text_column.split()  # Convierte la cadena en una lista de palabras
    except AttributeError:
        return []  # Si hay un error (por ejemplo, si es NaN), devuelve una lista vacía

In [15]:
# Aplico las transformaciones a las columnas correspondientes
df_mc['crew'] = df_mc['crew'].apply(extract_director)
df_mc['overview'] = df_mc['overview'].apply(overview_to_list)

In [16]:
print(df_mc[['crew', 'overview']].head())

                crew                                           overview
0      James Cameron  [In, the, 22nd, century,, a, paraplegic, Marin...
1     Gore Verbinski  [Captain, Barbossa,, long, believed, to, be, d...
2         Sam Mendes  [A, cryptic, message, from, Bond’s, past, send...
3  Christopher Nolan  [Following, the, death, of, District, Attorney...
4     Andrew Stanton  [John, Carter, is, a, war-weary,, former, mili...


In [17]:
# Elimino los espacios entre las palabras de las variables genres, cast, crew y keywords.
def quitar_espacios(lista):
    try:
        # Itero por cada elemento de la lista y elimino los espacios
        return [elemento.replace(' ', '') for elemento in lista]
    except TypeError:
        # En caso de error (si el valor no es lista), devolver el valor original
        return lista

In [18]:
# Aplico la función a las columnas genres, cast, crew y keywords.
df_mc['genres'] = df_mc['genres'].apply(quitar_espacios)
df_mc['cast'] = df_mc['cast'].apply(quitar_espacios)
df_mc['crew'] = df_mc['crew'].apply(quitar_espacios)
df_mc['keywords'] = df_mc['keywords'].apply(quitar_espacios)

In [19]:
print(df_mc[['genres', 'cast', 'crew', 'keywords']].head())

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, ScienceFiction]   

                                            cast  \
0  [SamWorthington, ZoeSaldana, SigourneyWeaver]   
1     [JohnnyDepp, OrlandoBloom, KeiraKnightley]   
2      [DanielCraig, ChristophWaltz, LéaSeydoux]   
3      [ChristianBale, MichaelCaine, GaryOldman]   
4    [TaylorKitsch, LynnCollins, SamanthaMorton]   

                                                crew  \
0             [J, a, m, e, s, , C, a, m, e, r, o, n]   
1          [G, o, r, e, , V, e, r, b, i, n, s, k, i]   
2                      [S, a, m, , M, e, n, d, e, s]   
3  [C, h, r, i, s, t, o, p, h, e, r, , N, o, l, a...   
4          [A, n, d, r, e, w, , S, t, a, n, t, o, n]   

                                           

In [20]:
# Combino todas las columnas procesadas en una sola columna llamada tags.
def crear_tags(fila):
    # Convierto en una cadena vacía si el valor es None o NaN
    overview = ' '.join(fila['overview']) if isinstance(fila['overview'], list) else str(fila['overview']) if pd.notnull(fila['overview']) else ''
    genres = ' '.join(fila['genres']) if isinstance(fila['genres'], list) else str(fila['genres']) if pd.notnull(fila['genres']) else ''
    cast = ' '.join(fila['cast']) if isinstance(fila['cast'], list) else str(fila['cast']) if pd.notnull(fila['cast']) else ''
    crew = ' '.join(fila['crew']) if isinstance(fila['crew'], list) else str(fila['crew']) if pd.notnull(fila['crew']) else ''
    keywords = ' '.join(fila['keywords']) if isinstance(fila['keywords'], list) else str(fila['keywords']) if pd.notnull(fila['keywords']) else ''
    
    # Combino todos los elementos en una sola cadena
    return overview + ' ' + genres + ' ' + cast + ' ' + crew + ' ' + keywords

# Aplico la función a cada fila del DataFrame para crear la columna 'tags'
df_mc['tags'] = df_mc.apply(crear_tags, axis=1)

In [21]:
print(df_mc[['tags']].head())

                                                tags
0  In the 22nd century, a paraplegic Marine is di...
1  Captain Barbossa, long believed to be dead, ha...
2  A cryptic message from Bond’s past sends him o...
3  Following the death of District Attorney Harve...
4  John Carter is a war-weary, former military ca...


In [22]:
# Elimino comas y las reemplazo por espacios en blanco (aunque no debería tener comas en este paso)
df_mc['tags'] = df_mc['tags'].str.replace(',', ' ')

In [23]:
print(df_mc['tags'].head())

0    In the 22nd century  a paraplegic Marine is di...
1    Captain Barbossa  long believed to be dead  ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary  former military ca...
Name: tags, dtype: object


4. Construir un KNN.

In [25]:
# Vectorizo el texto (para ello voy a usar el vectorizador "TfidfVectorizer")

# Inicio el vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)

# Vectorizo la columna 'tags'
vectors = vectorizer.fit_transform(df_mc['tags']).toarray()

# Verifico el tamaño del vector generado
print(vectors.shape) 

(4809, 5000)


In [27]:
# Calculo la distancia Coseno (mide la similitud entre dos vectores de características basándose en el ángulo entre ellos).

# Calculo la similitud coseno entre todas las películas.
similarity = cosine_similarity(vectors)

# Verifico la matriz de similitud.
print(similarity.shape) 

(4809, 4809)


In [28]:
# Función para recomendar películas (basada en la distancia de Coseno).
def recommend(movie):
    # Busco el índice de la película por su título
    movie_index = df_mc[df_mc["title"] == movie].index[0]
    
    # Obtengo las distancias de similitud entre la película dada y el resto
    distances = similarity[movie_index]
    
    # Ordeno las películas por similitud (excluyendo la propia película)
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Muestro las 5 películas más similares
    print(f"Películas similares a '{movie}':")
    for i in movie_list:
        print(df_mc.iloc[i[0]].title)


In [30]:
# Pruebo la función con un ejemplo
recommend("Pirates of the Caribbean: At World's End")


Películas similares a 'Pirates of the Caribbean: At World's End':
Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
The Pirates! In an Adventure with Scientists!
Life of Pi
